<a href="https://colab.research.google.com/github/Mehulsoni26/Uncertainty_Quantification_LLMs/blob/main/Code/Uncertainty_Quantification_phi_batch_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
import os
dir_path = '/content/drive/MyDrive'
os.chdir(dir_path)

In [ ]:
# !git clone https://github.com/Abhi23run/Uncertainty_Quantification_LLMs.git

In [4]:
os.chdir(dir_path+'/Uncertainty_Quantification_LLMs')

In [5]:
%%capture
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1
!pip install torch

In [6]:
import torch
import gc
import time
import evaluate
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import Dataset, load_dataset
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [7]:
df_stress_index = pd.read_csv('./Data/data_with_stress_index.csv',usecols=lambda col:col not in ['Unnamed: 0'])

In [8]:
n_bins = 10
bin_width = 1/n_bins

df_stress_index.loc[:,'stress_index_bucket']=pd.cut(df_stress_index['stress_index'],\
                                                    bins=n_bins,\
                                                    labels=[str(np.round(x,1))+'-'+str(np.round(x+bin_width,1)) \
                                                            for x in np.arange(0,1,bin_width)])

In [9]:
def shuffle_list(original_list):
    # Create a copy of the list
    shuffled_list = original_list.copy()

    # Shuffle the copy
    random.shuffle(shuffled_list)

    return shuffled_list

list_stress_index_bucket=[str(np.round(x,1))+'-'+str(np.round(x+bin_width,1)) for x in np.arange(0,1,bin_width)]
list_option_choices=[chr(x) for x in range(65,65+len(list_stress_index_bucket))]

In [10]:
import getpass

# Prompt for the Hugging Face token
hf_token = getpass.getpass("Enter your Hugging Face token: ")

import os
os.environ['HUGGINGFACE_TOKEN'] = hf_token

Enter your Hugging Face token: ··········


In [11]:
# model_id =  "NousResearch/Llama-2-7b-hf"
# model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "mistralai/Mistral-7B-v0.1"
model_id = "microsoft/phi-2"
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
# quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto",token=hf_token)

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_id,token=hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
choice_prompt=''''''
for i,j in zip(list_option_choices,shuffle_list(list_stress_index_bucket)):
    choice_prompt+=f'({i}) {j}\n'

In [14]:
def get_values_before_key(sorted_dict, key): ###updated the function to include the logits not including the true label
    values_before_key = []
    for k, v in sorted_dict.items():
        if k == key:
            # values_before_key.append(v)
            break
        values_before_key.append(v)
    return values_before_key

In [15]:
##Softmax function to convert logits to probabilities
def softmax(x):
    e_x=np.exp(x-np.max(x))
    return e_x/e_x.sum()

In [16]:
def lac(true_label, label_softmax_dict):
    lac_score = 1.0 - label_softmax_dict[true_label]
    return lac_score

In [18]:
def aps(true_label, label_softmax_dict):
    sorted_softmax_dict = dict(sorted(label_to_softmax_dict.items(), key=lambda item: item[1], reverse=True))
    high_labels = get_values_before_key(sorted_softmax_dict, true_label)
    aps_score = sum(high_labels)
    return aps_score

In [19]:
!ls

aps_score_list.pkl  Data		output_labels.pkl  predicion_set_aps.pkl  README.md
Code		    lac_score_list.pkl	output_logits.pkl  predicion_set_lac.pkl


In [20]:
df_stress_index_hf_prompt=pd.read_csv('/content/drive/MyDrive/Uncertainty_Quantification_LLMs/Data/df_stress_index_hf_prompt.csv',index_col=False,skiprows=[1]).head(2000)

###Instruction:
You are fixed income corporate bond trader. Use the feature values that I will provide and select the correct option for classifying the stress index (range of 0 to 1) into one of the defined buckets.
Feature descriptions:
- news_sentiment: Scale from -1 (negative) to +1 (positive) reflecting the sentiment in news about the bond issuer over the past 7 days.
- broker_count_imbalance: Difference in broker count buying or selling the same security in the last 2 days, ranging from -1 (more selling) to +1 (more buying).
- volume_indicator: Indicates if a bond is heavily traded in the last 2 days, with -1 for high selling volume and +1 for high buying volume.
- benchmark_price_difference: Compares a bond's quoted price to the benchmark, ranging from -1 (below benchmark) to +1 (above benchmark), indicating price stress.
- trade_count_imbalance: Difference in buy and sell trades of a security over 2 days, from -1 (more sells) to +1 (more buys).
- one_sided_trade_indicator: Imbalance in buy or sell trades over 7 days, with -1 for predominantly sell trades and +1 for buy trades.
- tranche_size_indicator: Assesses bond stress by the direction of trades and tranche size, ranging from 0 (large tranche) to 1 (small tranche).

###Question:
Given the feature value dict as:
{'news_sentiment': 0.3931444486731639, 'broker_count_imbalance': 0.1243619871996783, 'volume_indicator': 0.6778892697073087, 'benchmark_price_difference': 0.4977762417709135, 'trade_count_imbalance': -0.7206982745760815, 'one_sided_trade_indicator': -0.6800763028141159, 'tranche_size_indicator': 0.7248691201811168}

Perform 3 tasks as mentioned below.

Task 1: Think step-by-step and generate a concise bulleted reasoning chain to calculate the stress index on a scale of 0 to 1 using the feature value dict above. Output the reasoning steps within <stress_reason>..</stress_reason> tag.

Task 2 : Predict the value of stress index on a scale of 0 to 1 within <stress_index>..</stress_index> tag based on the reasoning chain.

Task 3: Return the correct option for classifying the stress index from the following options inside the <stress_bucket>..</stress_bucket> tag.The selection should be purely based on the stress index value computed within the <stress_index>..</stress_index>tag in Task 1.

###Options:
(A) 0.0-0.1
(B) 0.7-0.8
(C) 0.2-0.3
(D) 0.8-0.9
(E) 0.9-1.0
(F) 0.1-0.2
(G) 0.3-0.4
(H) 0.5-0.6
(I) 0.6-0.7
(J) 0.4-0.5

In [ ]:
data_splits = {'test': df_stress_index_hf_prompt}
df_stress_index_hf_dataset = {split: Dataset.from_pandas(pd.DataFrame(df_stress_index_hf_prompt[['row_prompt','answer_label','stress_index_bucket']])) for split, data in data_splits.items()}
df_stress_index_hf_dataset_inference=df_stress_index_hf_dataset['test']


In [ ]:
df_stress_index_hf_dataset_inference

Dataset({
    features: ['row_prompt', 'answer_label', 'stress_index_bucket'],
    num_rows: 250
})

In [ ]:
def process_batch(batch):
    """
    Tokenizes a batch of text inputs.
    Args:
        batch: A dictionary with a key 'text' containing a list of text inputs.
    Returns:
        A dictionary with keys 'input_ids', 'attention_mask' (and possibly others like 'token_type_ids' depending on the tokenizer).
    """
    # Tokenize the text inputs in the batch. This automatically applies padding and truncation.
    # Adjust `padding` and `truncation` as needed.
    tokenized_inputs = tokenizer(batch["row_prompt"], padding=True, truncation=True, return_tensors="pt")

    return tokenized_inputs

def test_data_collator(batch):
  input_ids=torch.stack([example['input_ids'] for example in batch])
  attention_mask=torch.stack([example['attention_mask'] for example in batch])

  return {'input_ids':input_ids, 'attention_mask':attention_mask}

In [ ]:
tokens_of_interest= list_option_choices #the option choices corresponding to stress index buckets

token_indices = tokenizer.convert_tokens_to_ids(tokens_of_interest)

In [ ]:
token_indices

[32, 33, 34, 35, 36, 37, 38, 39, 40, 41]

In [ ]:
batch_size=4
from torch.utils.data import DataLoader
from tqdm import tqdm

df_stress_index_hf_dataset_batched=df_stress_index_hf_dataset_inference.map(process_batch,batched=True,batch_size=batch_size)
df_stress_index_hf_dataset_batched.set_format(type='torch')

test_dataloader=DataLoader(
    df_stress_index_hf_dataset_batched,
    batch_size=batch_size,
    collate_fn=test_data_collator
)

output_labels=[]
for batch in tqdm(test_dataloader):
  input_ids=batch['input_ids']
  attention_mask=batch['attention_mask']


  with torch.no_grad():
    result=model(input_ids=input_ids,attention_mask=attention_mask,return_dict=True)
    next_token_logits=result.logits.detach().cpu()[:,-1,:]
    indices_in_logits = [{token: next_token_logits[i,token_idx].item() for token, token_idx in zip(tokens_of_interest, token_indices)} for i in range(next_token_logits.shape[0])]
    label_to_softmax_dict=[dict(zip(tokens_of_interest,softmax(np.array(list(indices_in_logits[i].values()))))) for i in range(len(indices_in_logits))]
    output_labels.append(label_to_softmax_dict)
    del input_ids
    del attention_mask
    del result
    gc.collect()
    torch.cuda.empty_cache()

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

100%|██████████| 500/500 [19:40<00:00,  2.36s/it]


In [ ]:
output_labels=[item for sublist in output_labels for item in sublist]

In [ ]:
import pickle
with open('./output_labels.pkl','wb') as f:
  pickle.dump(output_labels,f)

In [ ]:
with open('./output_labels.pkl','rb') as f:
  output_labels=pickle.load(f)

In [ ]:
lac_score_list=[]
aps_score_list=[]

##First 1500 rows as calibration dataset
for i,j in zip(output_labels[:1500],df_stress_index_hf_dataset_inference['answer_label'][:1500]):
  lac_score_list.append(lac(j,i))
  aps_score_list.append(aps(j,i))

In [ ]:
with open('./lac_score_list.pkl','wb') as f:
  pickle.dump(lac_score_list,f)

with open('./aps_score_list.pkl','wb') as f:
  pickle.dump(aps_score_list,f)

In [ ]:
with open('./lac_score_list.pkl','rb') as f:
  lac_score_list=pickle.load(f)

with open('./aps_score_list.pkl','rb') as f:
  aps_score_list=pickle.load(f)

In [ ]:
n=len(lac_score_list)

# user specified error rate
alpha=0.2
import math

def calculate_threshold(data,alpha):
    sorted_data = sorted(data)
    n = len(sorted_data)
    percentile = math.ceil(((n + 1)*(1-alpha)))/(n)  # Subtracting 1 because list indices start at 0
    # index = min(index, N - 1)  # Ensure the index does not exceed the last index of the list
    return np.percentile(sorted_data, percentile*100)

In [ ]:
lac_threshold=calculate_threshold(lac_score_list,alpha)

In [ ]:
aps_threshold=calculate_threshold(aps_score_list,alpha)

In [ ]:
tokens_of_interest

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

In [ ]:
predicion_set_lac=[]
for i_test in (output_labels[1500:]):
  predicion_set_i_lac=[]
  for token in tokens_of_interest:
    if lac(token,i_test)<=lac_threshold:
      predicion_set_i_lac.append(token)
  predicion_set_lac.append(predicion_set_i_lac)

prediction_set_aps=[]

for i_test in (output_labels[1500:]):
  predicion_set_i_aps=[]
  for token in tokens_of_interest:
    if aps(token,i_test)<=aps_threshold:
      predicion_set_i_aps.append(token)
  prediction_set_aps.append(predicion_set_i_aps)

In [ ]:
with open('./predicion_set_lac.pkl','wb') as f:
  pickle.dump(predicion_set_lac,f)

with open('./predicion_set_aps.pkl','wb') as f:
  pickle.dump(prediction_set_aps,f)

In [ ]:
with open('./predicion_set_lac.pkl','rb') as f:
  predicion_set_lac=pickle.load(f)

with open('./predicion_set_aps.pkl','rb') as f:
  prediction_set_aps=pickle.load(f)

Inference

In [46]:
import re
def extract_options(prompt):
    # Regex pattern to match the options
    pattern = r"\((\w)\) (\d\.\d-\d\.\d)"

    # Find all matches in the prompt
    matches = re.findall(pattern, prompt)

    # Create a dictionary to map letters to numeric ranges
    options_dict = {letter: range_ for letter, range_ in matches}

    return options_dict

# Function to extract the bucket based on the answer
def extract_bucket(answer, options_dict):
    # Return the numeric range corresponding to the answer
    return options_dict.get(answer, "Range not found.")

In [47]:
prompt=df_stress_index_hf_prompt.loc[2,'row_prompt']
answer=df_stress_index_hf_prompt.loc[2,'answer_label']
actual_bucket=df_stress_index_hf_prompt.loc[2,'stress_index_bucket']
input_ids = tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()
outputs = model.generate(input_ids=input_ids,
            temperature=0,
            do_sample=False,
            max_new_tokens=1,
        )
output= tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

# Extract all options from the prompt
options_dict = extract_options(prompt)
output_bucket = extract_bucket(output, options_dict)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [48]:
output,output_bucket,answer,actual_bucket

('E', '0.9-1.0', 'F', '0.5-0.6')